In [5]:
from crewai import Agent, Task, Crew
from crewai.llm import LLM
import os

# System Description for Tour Online Reservation System (TORS)
system_description = """
The Tour Online Reservation System (TORS) supports a travel agency by maintaining tour information and providing reservation facilities for people who wish to travel on tours by accessing a built-in network.
The system allows users to view information about the tours available and make a reservation on a tour without asking the employees at the agency.
Using the system, a user can cancel an existing reservation that the user made.
The system also allows a user to send feedbacks by e-mail to the agency and stores the feedback in the database.
Finally, the system allows the employees of the agency to manage customers and tours such as adding, deleting, and updating information on customers and tours.
For security purposes, the employees are required to provide login credentials (login ID and password) to access the system.
"""

# Ollama LLM Connection (example)
llm = LLM(
    model="ollama/llama3.2",
    base_url="http://localhost:11434",
    stream=True  # Enable streaming mode    
)

# Define Agents
requirements_identifier_agent = Agent(
    name="Requirements Identifier",
    role="Requirements Identification Expert",
    goal="Identify and categorize precise, atomic, and clearly distinguished functional and non-functional system requirements based on provided system descriptions.",
    backstory="A highly analytical requirements specialist, experienced in defining explicit, atomic, and clearly categorized system requirements that accurately address both functional and non-functional system aspects, ensuring consistency and clarity.",
    llm=llm,
    verbose=True
)

use_case_modeler_agent = Agent(
    name="Use Case Modeler",
    role="Use Case Modeling Expert",
    goal="Develop comprehensive and clear use case models, accurately defining system boundaries, actors, use cases, and their logical interactions to ensure all requirements are effectively captured and represented.",
    backstory="An accomplished use case modeling expert known for meticulously creating use case diagrams that clearly illustrate complete system functionalities and actor interactions, accurately aligning them with system requirements.",
    llm=llm,
    verbose=True
)

use_case_specification_agent = Agent(
    name="Use Case Specification Expert",
    role="Use Case Specification Specialist",
    goal="Write detailed, clear, and logically structured use case specifications, accurately describing actor-system interactions, primary and alternative scenarios, and pre/post conditions for each identified use case.",
    backstory="A dedicated expert in use case specification with substantial experience producing systematic, clear, and structured documentation of interactions and scenarios, effectively bridging the gap between analysis and detailed system design.",
    llm=llm,
    verbose=True
)

domain_modeling_agent = Agent(
    name="Domain Modeling Expert",
    role="Domain Model Specialist",
    goal="Create precise and concise domain models clearly representing meaningful domain concepts, relationships, multiplicities, and attributes based on detailed use case specifications, without introducing design-specific decisions.",
    backstory="A proficient domain modeling specialist with deep expertise in translating use case specifications into clear domain representations, accurately modeling real-world concepts and their interrelations while maintaining abstraction suitable for early system modeling phases.",
    llm=llm,
    verbose=True
)

system_operations_agent = Agent(
    name="System Operations Expert",
    role="System Operations Specialist",
    goal="Identify and define clear and accurate system operations, providing correct operation signatures with specified parameters and return types, derived systematically from use case specifications.",
    backstory="A highly skilled system operations analyst, expert at extracting clear operation definitions from use case details, ensuring precision in operation signatures, parameter definitions, and expected outcomes to effectively guide system design and implementation phases.",
    llm=llm,
    verbose=True
)

sequence_diagram_agent = Agent(
    name="Sequence Diagram Expert",
    role="Sequence Diagram Specialist",
    goal="Develop detailed, logical, and accurate design sequence diagrams that precisely depict object interactions, message flows, and collaborations to successfully realize identified system operations.",
    backstory="An expert sequence diagram designer skilled in accurately modeling object lifelines, interactions, and messaging sequences to effectively illustrate logical workflows and operational fulfillment within software systems.",
    llm=llm,
    verbose=True
)

design_class_diagram_agent = Agent(
    name="Design Class Diagram Expert",
    role="Design Class Diagram Specialist",
    goal="Create coherent and comprehensive design class diagrams, clearly defining classes, attributes, operations, relationships, navigabilities, visibilities, data types, and adhering strictly to design principles such as encapsulation and cohesion.",
    backstory="A highly experienced class diagram specialist known for systematically translating domain and sequence diagrams into robust, maintainable class structures, skillfully applying software design best practices and ensuring structural clarity.",
    llm=llm,
    verbose=True
)

implementation_agent = Agent(
    name="Implementation Expert",
    role="Implementation Specialist",
    goal="Develop robust and accurate software implementations strictly aligned with specified design class and sequence diagrams, ensuring adherence to defined method signatures, relationships, object interactions, visibility rules, and logical algorithms.",
    backstory="An accomplished software implementation specialist with extensive experience translating detailed design diagrams into syntactically correct, logically structured, and robust Java code, committed to precise implementation, secure application structures, and adherence to designed specifications.",
    llm=llm,
    verbose=True
)

testing_agent = Agent(
    name="Testing Specialist",
    role="Testing Expert",
    goal="Execute thorough and comprehensive software testing, including unit, integration, and system tests, ensuring full alignment with implementation requirements, accurate object interactions, and system behavior correctness.",
    backstory="A meticulous testing professional with significant expertise in designing, structuring, and executing detailed test scenarios across multiple levels—unit, integration, and system—consistently ensuring system reliability, functionality accuracy, and validation of specified input and output criteria.",
    llm=llm,
    verbose=True
)

# Task Definitions with Context
task1 = Task(
    description=f"""
    Using the following system description, identify functional and non-functional requirements: {system_description}. 
    Categorize functional requirements by features and non-functional requirements by quality attributes. Each requirement must be uniquely identified.
    Consider the following domain knowledge during analysis:
    - Functional requirements describe specific functions the system must perform, while non-functional requirements describe qualities the system must meet.
    Ensure the use case model accurately reflects the functional requirements as specific system actions and incorporates non-functional requirements as quality constraints influencing the actors and use cases.
    """,
    agent=requirements_identifier_agent,
    expected_output="A clearly organized list of uniquely identified functional and non-functional requirements categorized by relevant features and quality attributes."
)

task2 = Task(
    description=f"""
    Using the list of functional and non-functional requirements identified in the previous task, create a use case model. Clearly represent the system boundary, actors, and their connections to use cases. Include proper relationship notation and assign unique identifiers to each use case.
    Consider the following domain knowledge during modeling:
    - Use case modeling organizes system functionalities through use cases (describing system actions to deliver results to actors), actors (who interact with the system), and system boundaries (defining the scope), and utilizes associations to connect actors with use cases.
    Ensure the use case specifications align with this structure, detailing how each use case delivers results to actors within the defined system boundaries and through specified associations.
    """,
    agent=use_case_modeler_agent,
    expected_output="A comprehensive use case model diagram clearly depicting system boundaries, actor interactions, relationships, and uniquely identified use cases.",
    context=[task1]  # Depends on task1’s output
)

task3 = Task(
    description=f"""
    Using the use case model created in the previous task, develop detailed use case specifications for each identified use case. Each specification must include Use Case ID, Use Case Name, Primary Actor, Preconditions, Postconditions, Main Flow, and Alternative Flows, clearly illustrating interactions between actors and the system.
    Consider the following domain knowledge during specification:
    - A fully dressed use case specification is a comprehensive document that details all aspects of a system interaction, covering the main scenario, alternative flows, preconditions, postconditions, stakeholder interests, and special requirements to ensure a clear and complete description of the system's functionality from an end-user perspective. The main scenario outlines the interactions between the primary actor initiating system operations and the system responding to each action.
    Ensure the domain model accurately reflects the entities, relationships, and attributes derived from the main scenarios and alternative flows of the use case specifications, capturing the system's functionality as described from the end-user perspective.
    """,
    agent=use_case_specification_agent,
    expected_output="Detailed use case specifications accurately documenting actor-system interactions, pre/post conditions, main flows, and alternative scenarios for all use cases.",
    context=[task2]  # Depends on task2’s output
)

task4 = Task(
    description=f"""
    Based on the detailed use case specifications from the previous task, create a clear and concise domain model. Represent classes, relationships, multiplicities, and attributes using proper notation. Avoid including design-specific details such as visibility, navigability, and data types.
    Consider the following domain knowledge during modeling:
    - Use case specifications are used for domain modeling to identify necessary domain model constituents, such as domain classes (domain concepts), attributes, and relationships, through detailed functional scenarios. Domain concepts are typically identified as nouns representing complex entities with their own attributes in the use case specifications. Attributes are also identified as nouns, but they usually represent simple data types, such as strings and numbers, but they must not be typed.
    Leverage this knowledge to extract system operations from the functional scenarios in the use case specifications, ensuring operations correspond to actions on domain concepts and their attributes, while specifying parameters and return types appropriately.
    """,
    agent=domain_modeling_agent,
    expected_output="A precise domain model diagram accurately reflecting core domain concepts, their attributes, and relationships without specific design details.",
    context=[task3]  # Depends on task3’s output
)

task5 = Task(
    description=f"""
    Using the detailed use case specifications from the previous task, identify system operations. Provide each operation with the correct signature format (operationName(parameter1: parameterType, ...): returnType), clearly specifying parameter types and return types.
    Consider the following domain knowledge during operation identification:
    - System operations are identified from use case specifications through a detailed analysis of interactions between actors and the system described in the use cases. Each user action (not system response) in a use case corresponds to system operations, which are services provided at the interface level. These operations, directly derived from the user actions outlined in the main success scenarios of the use cases, enable actors to initiate specific system functions.
    Ensure the sequence diagrams accurately depict these system operations as interactions initiated by actors, with lifelines representing objects that execute the operations and messages reflecting the user actions from the main success scenarios.
    """,
    agent=system_operations_agent,
    expected_output="A systematically derived list of system operations with accurate signatures, parameters, and return types aligned with use case interactions.",
    context=[task3]  # Depends on task3’s output
)

task6 = Task(
    description=f"""
    Using the system operations identified in the previous task, create design sequence diagrams. Lifelines must represent existing objects, interactions must include accurate message notation, and clearly depict return messages when appropriate.
    Consider the following domain knowledge during diagram creation:
    - For each system operation, a sequence diagram is created to illustrate how the operation is executed through the collaboration of objects within the system. This process involves identifying participants and detailing their interactions through message exchanges. Each participant is an object of a class derived from the domain model.
    - It is crucial to note that the initiator of the system operation is the primary actor of the use case in which the operation is defined. This actor should not be included as a participant in the sequence diagram, as it is external to the system. Instead, the initiating system operation should be depicted as a found message at the beginning of the sequence diagram.
    - For example, if a system operation s() is initiated by actor P, s() is represented as a found message (the first message) received by a participant within the sequence diagram. If the receiving participant is an object of class A, the class A is derived from the corresponding domain class A in the domain model and the class A is assigned the operation s() (s() is defined in the class A).
    - Then, the operation s() is performed in a sequence of messages among objects of various classes within the system. For example, the receiving object of class A may send a message m1() to an object of class B, which then sends a message m2() to an object of class C, and so on. This assigns m1() to class B and m2() to class C.
    Ensure the design class diagram accurately defines classes with operations derived from the sequence diagrams, assigning each operation (e.g., s(), m1(), m2()) to the appropriate class based on the message exchanges, while integrating domain classes from the domain model and adhering to design principles.
    """,
    agent=sequence_diagram_agent,
    expected_output="Accurate design sequence diagrams clearly illustrating object interactions, message flows, and object collaborations for system operations.",
    context=[task5]  # Depends on task5’s output
)

task7 = Task(
    description=f"""
    Using the domain model from task 4 and the sequence diagrams from the previous task, develop a design class diagram. Include classes with attributes and operations, proper relationships, visibility indicators, method signatures, navigabilities, and data types, adhering to good design principles such as encapsulation and cohesion.
    Consider the following domain knowledge during class diagram development:
    - Design class diagrams are developed based on the domain class diagram (domain model) and design sequence diagrams. The domain class diagram informs the attributes and class relationships in design class diagrams. Based on the attributes in the domain model, design class diagrams determine the type of attributes. Design class diagrams must be consistent with design sequence diagrams, which determine operation assignments and the navigability of class relationships.
    - For example, if an instance of class A calls a message m() on an instance of class B, the operation m() must be defined in class B, not class A. This also determines the navigability of the relationship between class A and class B, ensuring that the relationship end on class B is navigable.
    Ensure the design class diagram integrates attributes and relationships from the domain model, assigns operations to classes based on the sequence diagrams' message exchanges (e.g., placing m() in the called class), determines attribute types, and sets navigability consistent with the interactions, while adhering to design principles.    
    """,
    agent=design_class_diagram_agent,
    expected_output="A detailed, comprehensive design class diagram accurately reflecting class structures, relationships, operations, and design principles.",
    context=[task4, task6]  # Depends on task4 and task6’s outputs
)

task8 = Task(
    description=f"""
    Using the design class diagram and sequence diagrams from the previous task, develop a robust Java implementation. Code must be syntactically correct, with classes structured appropriately (constructors, setters, getters) and methods strictly aligned with design signatures, algorithms, and relationships.
    Consider the following domain knowledge during implementation:
    - The implementation must conform to class diagrams for defining the structure and to sequence diagrams for dictating behaviors. This means that the structure of the system, including the classes, their attributes, methods, and relationships (with navigabilities and multiplicities), should be implemented as specified in the class diagrams. Similarly, the behaviors — how objects interact through sequences of operations and messages — should match the flow and details provided in the sequence diagrams.
    Ensure the Java code accurately reflects the class diagram’s structure (e.g., classes, attributes, methods, and relationships) and implements the behavioral flow of operations as depicted in the sequence diagrams, maintaining consistency between structure and behavior.
    """,
    agent=implementation_agent,
    expected_output="Fully functional, syntactically correct Java implementation strictly aligned with provided class and sequence diagrams, accurately reflecting specified algorithms and interactions.",
    context=[task7]  # Depends on task7’s output
)

task9 = Task(
    description=f"""
    Based on the Java implementation from the previous task, develop a comprehensive test suite. Include unit tests verifying individual methods, integration tests validating class interactions, and system tests covering complete use case scenarios. Provide clearly formatted test methods, structured assertions, and appropriate test setup and teardown procedures.
    Consider the following domain knowledge during test development:
    - Tests including unit tests, integration tests, and system tests should be developed to validate software at different levels. Unit tests should cover all methods in all classes to ensure individual functionality, while integration tests should target key interactions depicted in sequence diagrams to verify component interoperability. System tests, based on the overall requirements, should confirm that the entire system functions as intended in its operational environment and should exercise the complete scenarios captured in each sequence diagram.
    Ensure the test suite includes unit tests for all class methods, integration tests for sequence diagram interactions, and system tests that validate the full scenarios against the system requirements, with clear assertions and proper setup/teardown routines.
    """,
    agent=testing_agent,
    expected_output="A thorough test suite including unit, integration, and system tests, clearly structured with formatted assertions, method signatures, setup and teardown routines, and comprehensive test results.",
    context=[task8]  # Depends on task8’s output
)

# Create Crew
crew = Crew(
    agents=[
        requirements_identifier_agent, 
        use_case_modeler_agent, 
        use_case_specification_agent, 
        domain_modeling_agent, 
        system_operations_agent,
        sequence_diagram_agent,
        design_class_diagram_agent,
        implementation_agent,
        testing_agent
    ],
    tasks=[
        task1, task2, task3, task4, task5, task6, task7, task8, task9
    ],
    process="sequential",  # Ensure tasks are executed in order
    full_output=True,      # Enable full output for all tasks
    verbose=True           # Show detailed execution logs    
)

# Run Crew
result = crew.kickoff()

# Print detailed outputs for all tasks
for i, task in enumerate(crew.tasks):
    print(f"Task {i+1}: {task.description}")
    print(f"Raw Output: {task.output.raw}")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 3efdcfa6-1cd1-452d-bd97-bf29600c585d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: fc4b75e5-7151-4642-8226-6d0dbb7894ab
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: fc4b75e5-7151-4642-8226-6d0dbb7894ab
       Status: Executing Task...
    └── 🤖 Agent: Requirements Identification Expert
            Status: In Progress

# Agent: Requirements Identification Expert
## Task: 
    Using the following system description, identify functional and non-functional requirements: 
The Tour Online Reservation System (TORS) supports a travel agency by maintaining tour information and providing reservation facilities for people who wish to travel on tours by accessing a built-in network.
The system allows users to view information about the tours available and make a reservation on a tour without asking the employees at the agency.
Using the system, a user can cancel an existing reservation that the user made.
The system also allows a user to send feedbacks by e-mail to the agency and stores the feedback in the database.
Finally, the system allows the employees of the agency to manage customers and tours such as adding, deleting, and updating information on customers and tours.
For security purposes, the employees are required to provide login credentials (login ID and password) to access the system.
. 
    Categori

🚀 Crew: crew
└── 📋 Task: fc4b75e5-7151-4642-8226-6d0dbb7894ab
       Status: Executing Task...
    └── 🤖 Agent: Requirements Identification Expert
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: fc4b75e5-7151-4642-8226-6d0dbb7894ab
│      Assigned to: Requirements Identification Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Requirements Identification Expert
│           Status: ✅ Completed
├── 📋 Task: cb0c073d-e09d-483a-8974-e5daad83288f
│      Assigned to: Use Case Modeling Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Use Case Modeling Expert
│           Status: ✅ Completed
├── 📋 Task: 1d0c43ec-c0bb-4bbe-bdc4-286c997e9f5c
│      Assigned to: Use Case Specification Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Use Case Specification Specialist
│           Status: ✅ Completed
├── 📋 Task: 83cc2e0e-17fb-40ae-bc81-91f9094171cf
│      Assigned to: Domain Model Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Domain Model Specialist
│           Status: ✅ Completed
├── 📋 Task: 97a396e1-bee0-4f95-adb9-f50a9a807dc7
│      Assigned to: System Operations Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: System Operations Specialist
│           Status: ✅ Completed
├── 📋 Task: 5462e136-c02d-4bdf-8a27-7ee3e0cf381b
│      Assigned to: Sequence Diagram Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Sequence Diagram Specialist
│           Status: ✅ Completed
├── 📋 Task: 7a8d63ec-d238-48c9-90e1-89f6425b7bd5
│      Assigned to: Design Class Diagram Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Design Class Diagram Specialist
│           Status: ✅ Completed
├── 📋 Task: fe7e3161-ad00-4bb6-96bd-026bb8aeffd6
│      Assigned to: Implementation Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Implementation Specialist
│           Status: ✅ Completed
└── 📋 Task: aaae9eea-5909-4c1b-9280-210309af7b67
       Status: Executing Task...
    └── 🤖 Agent: Testing Expert
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: fc4b75e5-7151-4642-8226-6d0dbb7894ab
│      Assigned to: Requirements Identification Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Requirements Identification Expert
│           Status: ✅ Completed
├── 📋 Task: cb0c073d-e09d-483a-8974-e5daad83288f
│      Assigned to: Use Case Modeling Expert
│      Status: ✅ Completed
│   └── 🤖 Agent: Use Case Modeling Expert
│           Status: ✅ Completed
├── 📋 Task: 1d0c43ec-c0bb-4bbe-bdc4-286c997e9f5c
│      Assigned to: Use Case Specification Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Use Case Specification Specialist
│           Status: ✅ Completed
├── 📋 Task: 83cc2e0e-17fb-40ae-bc81-91f9094171cf
│      Assigned to: Domain Model Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Domain Model Specialist
│           Status: ✅ Completed
├── 📋 Task: 97a396e1-bee0-4f95-adb9-f50a9a807dc7
│      Assigned to: System Operations Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: System Operations Specialist
│           Status: ✅ Completed
├── 📋 Task: 5462e136-c02d-4bdf-8a27-7ee3e0cf381b
│      Assigned to: Sequence Diagram Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Sequence Diagram Specialist
│           Status: ✅ Completed
├── 📋 Task: 7a8d63ec-d238-48c9-90e1-89f6425b7bd5
│      Assigned to: Design Class Diagram Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Design Class Diagram Specialist
│           Status: ✅ Completed
├── 📋 Task: fe7e3161-ad00-4bb6-96bd-026bb8aeffd6
│      Assigned to: Implementation Specialist
│      Status: ✅ Completed
│   └── 🤖 Agent: Implementation Specialist
│           Status: ✅ Completed
└── 📋 Task: aaae9eea-5909-4c1b-9280-210309af7b67
       Assigned to: Testing Expert
       Status: ✅ Completed
    └── 🤖 Agent: Testing Expert
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: aaae9eea-5909-4c1b-9280-210309af7b67                                                                     │
│  Agent: Testing Expert                                                                                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 3efdcfa6-1cd1-452d-bd97-bf29600c585d                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Task 1: 
    Using the following system description, identify functional and non-functional requirements: 
The Tour Online Reservation System (TORS) supports a travel agency by maintaining tour information and providing reservation facilities for people who wish to travel on tours by accessing a built-in network.
The system allows users to view information about the tours available and make a reservation on a tour without asking the employees at the agency.
Using the system, a user can cancel an existing reservation that the user made.
The system also allows a user to send feedbacks by e-mail to the agency and stores the feedback in the database.
Finally, the system allows the employees of the agency to manage customers and tours such as adding, deleting, and updating information on customers and tours.
For security purposes, the employees are required to provide login credentials (login ID and password) to access the system.
. 
    Categorize functional requirements by features and no